In [4]:
include("../src/cvodes.jl")

function f(t,y,p,dy)
    dy[1] = p[1] + 1
    dy[2] = p[2]^2 * y[1]
end

@time Sensivity.cvodes(f, [0., 0], [0., 1], collect(0:1.:10), autodiff = true)

  0.487958 seconds (319.38 k allocations: 14.363 MB, 5.17% gc time)


(
11x2 Array{Float64,2}:
  0.0   0.0
  1.0   0.5
  2.0   2.0
  3.0   4.5
  4.0   8.0
  5.0  12.5
  6.0  18.0
  7.0  24.5
  8.0  32.0
  9.0  40.5
 10.0  50.0,

11x2x2 Array{Float64,3}:
[:, :, 1] =
  0.0   0.0
  1.0   0.5
  2.0   2.0
  3.0   4.5
  4.0   8.0
  5.0  12.5
  6.0  18.0
  7.0  24.5
  8.0  32.0
  9.0  40.5
 10.0  50.0

[:, :, 2] =
 0.0            0.0
 0.0            1.0
 0.0            4.0
 1.17882e-23    9.0
 3.43406e-23   16.0
 3.83905e-23   25.0
 3.55456e-23   36.0
 3.05636e-23   49.0
 2.34445e-23   64.0
 1.41883e-23   81.0
 1.54483e-23  100.0)

In [176]:
using GynC
include("../src/gyncycle.jl")

p = copy(GynC.refallparms)
y0 = copy(GynC.refy0)

dy = Array(Float64,33)
D! = differentiator((t, y, p, dy) -> gyncycle_rhs!(y, p, dy), 33, 114);

In [404]:
ind = [3]

yscales = maximum(sol_cvode[:,:], 1) |> vec
tempp = convert(Vector{Any}, p)

function fill(pp, ind)
    allp[ind] = pp    
    allp::Vector{Any}
end

t=[0:1.:31]

function scaledsystem(t,ys,ps,dy)
    y = ys .* yscales
    tempp[ind] = ps .* p[ind]
    gyncycle_rhs!(y, tempp, dy)
    dy[:] = dy[:] ./ yscales
    dy
end

@time sol_cvodes, sens = cvodes(scaledsystem, y0./yscales, ones(length(ind)), t, autodiff=true, stiff=true, solvertype=:dense, reltol = 1e-5, abstol=1e-5, reltolS = 1e-4, abstolS = 1e-5, errconS = 1)

sens[1:26,:,:]

  0.678268 seconds (3.72 M allocations: 281.786 MB, 20.17% gc time)



[CVODES ERROR]  CVode
  At t = 15.572 and h = 9.88467e-09, the error test failed repeatedly or with |h| = hmin.



26x33x1 Array{Float64,3}:
[:, :, 1] =
  0.0           0.0            0.0         …     0.0           0.0        
 -3.76864e-7   -1.01809e-8     4.5384e-8         1.18825e-10   1.0822e-10 
 -4.34383e-7   -1.19773e-8     5.33068e-8        9.57076e-10   9.16658e-10
 -4.46221e-7   -1.31076e-8     5.73344e-8        2.79632e-9    2.71932e-9 
 -4.49225e-7   -1.50803e-8     6.32409e-8        5.72034e-9    5.59882e-9 
 -4.50339e-7   -1.75866e-8     6.91244e-8  …     9.96842e-9    9.78678e-9 
 -4.52661e-7   -1.88984e-8     6.97665e-8        1.60022e-8    1.57404e-8 
 -4.62752e-7   -1.81647e-8     6.46939e-8        2.35568e-8    2.32462e-8 
 -5.0443e-7    -1.81959e-8     6.42461e-8        2.97716e-8    2.95238e-8 
 -6.59649e-7   -2.50508e-8     8.88423e-8        3.23727e-8    3.22216e-8 
 -1.17663e-6   -5.15063e-8     1.83875e-7  …     3.02581e-8    3.0326e-8  
 -2.8065e-6    -1.20706e-7     4.38108e-7        6.37157e-8    5.53278e-8 
 -8.14779e-6   -4.20884e-7     1.49649e-6       -4.43193e-8   

In [299]:
# sundials vs limex
@time sol_cvode = Sundials.cvode((t, y, dy) -> gyncycle_rhs!(y, p, dy), y0, t) #, reltol = 1e-2, abstol=1e-2)
@time sol_limex = GynC.gync(y0, t, p);

  0.011321 seconds (6.45 k allocations: 197.322 KB)
  0.042094 seconds (6 allocations: 8.875 KB)


In [284]:
# compare solutions
using PyPlot
species = 7
PyPlot.plot(t, sol_cvode[:,species], "<", t, sol_cvodes[:,species], ">", t, sol_limex[species,:] |> vec, "^") 

3-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7fbfebd99d10>
 PyObject <matplotlib.lines.Line2D object at 0x7fbfebd99dd0>
 PyObject <matplotlib.lines.Line2D object at 0x7fbfebe8de10>